In [1]:
# import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
import datasets

# Load the dataset
dataset = datasets.load_dataset("GEM/xlsum", "arabic")

# Print the dataset info
print(dataset)

Found cached dataset xlsum (C:/Users/DELL/.cache/huggingface/datasets/GEM___xlsum/arabic/2.0.0/eb0c1bd988fe61962620fe73722ebc91e0fd5729b4c8acbf3e3b3c50f8b22a96)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['gem_id', 'url', 'title', 'target', 'references', 'text'],
        num_rows: 37519
    })
    test: Dataset({
        features: ['gem_id', 'url', 'title', 'target', 'references', 'text'],
        num_rows: 4689
    })
    validation: Dataset({
        features: ['gem_id', 'url', 'title', 'target', 'references', 'text'],
        num_rows: 4689
    })
})


### Discover how the dataset looks like

In [3]:
for i in range(3):
    print(dataset["train"][i])


{'gem_id': 'xlsum_arabic-train-1', 'url': 'https://www.bbc.com/arabic/worldnews/2014/03/140323_russian_troops_crimea_naval_base', 'title': 'القوات الأوكرانية تبدأ الانسحاب من القرم', 'target': 'بدأت القوات الأوكرانية الانسحاب من شبه جزيرة القرم. \n\n', 'references': ['بدأت القوات الأوكرانية الانسحاب من شبه جزيرة القرم. \n\n'], 'text': 'وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينوف، قد أمر بسحب جميع القوات الأوكرانية من القرم.\n\nوسيطرت قوات روسية صباح الاثنين على قاعدة بحرية أوكرانية في فيودوسيا، في ثالث هجوم من نوعه خلال 48 ساعة، وذلك بحسب تصريحات مسؤولين أوكرانيين لبي بي سي .\n\nوقال المتحدث باسم وزارة الدفاع الأوكرانية فلاديسلاف سيليزنيوف إن القوات الروسية هاجمت القاعدة وألقت القبض على الجنود الأوكرانيين في قاعدة فيودوسيا وقيدت أيادي ضباطهم.\n\nومن المتوقع أن تسيطر الأزمة الأوكرانية على قمة مجموعة الدول الصناعية السبع في لاهاي.\n\nمواضيع قد تهمك نهاية\n\nوأكد الرئيس الأمريكي باراك أوباما خلال لقاء مع نظيره الصيني شى جين بينغ على أن "واشنطن وبكين يمكنهما، بالعمل سويا، تعزيز القانون

In [4]:
import pandas as pd

### From the above, we are interested in the 'text' and 'target' tags

In [5]:
train_inputs = dataset['train']['text']
train_labels = dataset['train']['target']


In [6]:
train_inputs = pd.DataFrame(train_inputs)
train_labels = pd.DataFrame(train_labels)
# Print the shape of the inputs and labels
print("Train inputs shape:", train_inputs.shape)
print("Train labels shape:", train_labels.shape)

Train inputs shape: (37519, 1)
Train labels shape: (37519, 1)


## Text Cleaning

### Analyze text by checking if it has any punctuation symbols

In [7]:
punctuations= ['?',':','!','.',',',';','؟','؛','،','(',')','/','"','-','\\', '\n', '#','*']

# Function to count occurrences of punctuations in a given sentence
def count_punctuations(sentence):
    return sum([sentence.count(p) for p in punctuations])

# Add new column with punctuation counts for each sentence
train_inputs['Punctuation_Counts_before'] = train_inputs[0].apply(count_punctuations)
train_labels['Punctuation_Counts_before'] = train_labels[0].apply(count_punctuations)

# Calculate total punctuation count for all sentences in DataFrame
total_punctuation_count_train = train_inputs['Punctuation_Counts_before'].sum()
total_punctuation_count_label = train_labels['Punctuation_Counts_before'].sum()


# Print total punctuation count
print(f'Total Training Punctuation Count: {total_punctuation_count_train}')
print(f'Total Label Punctuation Count: {total_punctuation_count_label}')

Total Training Punctuation Count: 3608886
Total Label Punctuation Count: 115670


### Normalize by removing all punctuations

In [8]:

replace_punc = lambda x: ''.join([' ' if ch in punctuations else ch for ch in x])
train_inputs.iloc[:, 0] = train_inputs.iloc[:, 0].apply(replace_punc)
train_labels.iloc[:, 0] = train_labels.iloc[:, 0].apply(replace_punc)


In [9]:
# Add new column with punctuation counts for each sentence
train_inputs['Punctuation_Counts_after'] = train_inputs[0].apply(count_punctuations)
train_labels['Punctuation_Counts_after'] = train_labels[0].apply(count_punctuations)

# Calculate total punctuation count for all sentences in DataFrame
total_punctuation_count_train = train_inputs['Punctuation_Counts_after'].sum()
total_punctuation_count_label = train_labels['Punctuation_Counts_after'].sum()


# Print total punctuation count
print(f'Total Training Punctuation Count: {total_punctuation_count_train}')
print(f'Total Label Punctuation Count: {total_punctuation_count_label}')

Total Training Punctuation Count: 0
Total Label Punctuation Count: 0


In [10]:
#Drop columns to avoid confusion
train_inputs.drop('Punctuation_Counts_after', axis=1, inplace=True)
train_labels.drop('Punctuation_Counts_after', axis=1, inplace=True)


### We will now analyze by checking one of the common problems in arabic datasets. The problem is the use of the letters 'ه' and 'ة' at the end of the words interchangeably. For example, the two words 'المستقبلية' and 'المستقبليه' may both appear in the dataset, represented by two different words although they should be the same word. So we start by checking if this problem indeed exists in our dataset.

### We collect all words ending with 'ه' and all word ending with 'ة' and check if there are common words.

In [11]:
def words_ending_with_character(string, character):
    # Split the string into individual words
    words = string.split()
    # Use a list comprehension to find words that end with the specified character
    result = [word for word in words if word.endswith(character)]
    return result

def count_of_words_ending_with(letter1='ه', letter2='ة'):
    #initialize temporary dataframe
    df = pd.DataFrame()
    #get all the words that end with ha2 marbota
    df['words_ending_with_ha2'] = train_inputs[0].apply(words_ending_with_character, character=letter1)
    # Compress all the words in the column into a single column
    all_words_with_ha2 = [word for sublist in df['words_ending_with_ha2'] for word in sublist]

    # Get the unique words in the list
    unique_words_with_ha2 = list(set(all_words_with_ha2))
    # Remove last character (ha2) in all strings in the unique_words list to compare the word with words ending with ta2
    unique_words_with_ha2 = [word[:-1] for word in unique_words_with_ha2]

    #initialize temporary dataframe
    df = pd.DataFrame()
    #get all the words that end with ta2 marbota
    df['words_ending_with_ta2'] = train_inputs[0].apply(words_ending_with_character, character=letter2)
    # Compress all the lists in the column into a single column
    all_words_with_ta2 = [word for sublist in df['words_ending_with_ta2'] for word in sublist]

    # Get the unique words in the list
    unique_words_with_ta2 = list(set(all_words_with_ta2))
    # Remove last character (ta2) in all strings in the unique_words list to compare the word with words ending with ta2
    unique_words_with_ta2 = [word[:-1] for word in unique_words_with_ta2]

    common_words = set(unique_words_with_ha2) & set(unique_words_with_ta2)

    print("Common words:", list(common_words))
    print("Number of words appearing with both",letter1, 'and', letter2, "in the end:", len(list(common_words)))

count_of_words_ending_with('ه','ة')

Common words: ['', 'نفق', 'احتياطي', 'الدول', 'القياد', 'عطل', 'الشيكولات', 'الروبي', 'تهم', 'للغ', 'قريب', 'حديث', 'موز', 'للموافق', 'واج', 'وثق', 'لصاحب', 'وحماس', 'الاستقال', 'خضير', 'بصور', 'وحشي', 'سنغافور', 'وتنقي', 'شالك', 'ناقص', 'شخصي', 'للمر', 'وتسمي', 'وقواعد', 'بملامح', 'لإصاب', 'تعزي', 'اولي', 'أطلقت', 'وشهر', 'الساع', 'وتذكر', 'يشتب', 'علاج', 'درن', 'بمقدم', 'لين', 'حر', 'وعزل', 'فالك', 'فل', 'بلاغ', 'بديل', 'بالديمقراطي', 'وخديج', 'لحملت', 'نخل', 'مواكب', 'ملموس', 'لملاحق', 'متوقف', 'لدغ', 'بحرق', 'طلح', 'حال', 'شرك', 'قدمي', 'مرسم', 'حد', 'كوري', 'بطلب', 'جلب', 'الجاي', 'لمخرج', 'قص', 'دار', 'أدعي', 'وطرف', 'الحاكم', 'مأمن', 'بمواقف', 'عقد', 'الجنين', 'بصحب', 'النصر_الوحد', 'سلف', 'صعوب', 'مديح', 'المنشا', 'تسع', 'بقسم', 'لكم', 'شفت', 'رحيل', 'عرض', 'وبلنسي', 'الكتروني', 'وآي', 'نطق', 'مجمل', 'انتقال', 'كن', 'ومترجم', 'ميز', 'لولا', 'الجنوبي', 'تجديد', 'اندهاش', 'دافع', 'متلقي', 'نكر', 'رفع_الحظر_عن_الملاعب_العراقي', 'الجماع', 'وأضاف', 'معد', 'عباي', 'لملك', 'ودعا', 'تل

### We do the same analysis with 'ي' and 'ى'

In [12]:
count_of_words_ending_with('ي', 'ى')

Common words: ['', 'ينتم', 'عُر', 'الدول', 'إنن', 'القياد', 'وينه', 'وتستول', 'ضواح', 'وادع', 'شور', 'تنف', 'طه', 'تقو', 'اغن', 'وتحظ', 'أهلاو', 'والقر', 'واشتر', 'سيعتد', 'ترق', 'العاط', 'ودو', 'سأنه', 'تفش', 'بالتخل', 'التقص', 'جيبوت', 'ورب', 'المرم', 'نخل', 'نرض', 'تشف', 'حال', 'اسم', 'ديمبل', 'النو', 'نه', 'وسيلغ', 'داع', 'ترتد', 'تختف', 'يستدع', 'يغط', 'أول', 'والحل', 'المثن', 'قص', 'أنتون', 'تعد', 'ارتد', 'أو', 'مغز', 'للموسيق', 'عقد', 'وايم', 'يرو', 'فرسا', 'تسع', 'يتنح', 'برونا', 'كيد', 'أفيخا', 'الجو', 'خب', 'المصطف', 'ويهو', 'الرسم', 'تسو', 'ويعف', 'الجماع', 'معتصم', 'وتحي', 'لاينبغ', 'الشربين', 'الحسين', 'وستجر', 'الناد', 'وسير', 'سلماو', 'والغن', 'وتحص', 'يتول', 'بالقو', 'مهاجم', 'ت', 'الكويت', 'مرض', 'تقض', 'ثمان', 'ذو', 'وتر', 'ورل', 'شوق', 'ومسيح', 'السكن', 'أدو', 'والتق', 'أغر', 'تقص', 'شرف', 'تحتف', 'المغل', 'ادن', 'وتستدع', 'المبان', 'ستبن', 'تعال', 'لاق', 'فألق', 'الأراض', 'ليقض', 'تأييد', 'مؤيد', 'تحص', 'الإيطال', 'تُخف', 'الاسد', 'دستور', 'جرح', 'وتنه', 'منتقد', 'و

### We notice that there is an inconsistent use of 'ة' and 'ه' at the end of the words. The same goes with the use of 'ي' and 'ى'.
### Thus, we need to normalize by replacing 'ة'' with 'ه' and 'ى' with 'ي'

In [13]:
replace_ha2_marboota = lambda x: ''.join(['ه' if ch == 'ة' else ch for ch in x])
replace_ya2 = lambda x: ''.join(['ي' if ch == 'ى' else ch for ch in x])
train_inputs.iloc[:,0] = train_inputs.iloc[:,0].apply(replace_ha2_marboota)
train_labels.iloc[:,0] = train_labels.iloc[:,0].apply(replace_ha2_marboota)
train_inputs.iloc[:,0] = train_inputs.iloc[:,0].apply(replace_ya2)
train_labels.iloc[:,0] = train_labels.iloc[:,0].apply(replace_ya2)


In [14]:
count_of_words_ending_with('ه','ة')
count_of_words_ending_with('ي', 'ى')

Common words: []
Number of words appearing with both ه and ة in the end: 0
Common words: []
Number of words appearing with both ي and ى in the end: 0


In [15]:
print(train_inputs[0][0])
print(train_labels[0][0])

وكان الرئيس الأوكراني المؤقت  الكسندر تورتشينوف  قد أمر بسحب جميع القوات الأوكرانيه من القرم   وسيطرت قوات روسيه صباح الاثنين علي قاعده بحريه أوكرانيه في فيودوسيا  في ثالث هجوم من نوعه خلال 48 ساعه  وذلك بحسب تصريحات مسؤولين أوكرانيين لبي بي سي    وقال المتحدث باسم وزاره الدفاع الأوكرانيه فلاديسلاف سيليزنيوف إن القوات الروسيه هاجمت القاعده وألقت القبض علي الجنود الأوكرانيين في قاعده فيودوسيا وقيدت أيادي ضباطهم   ومن المتوقع أن تسيطر الأزمه الأوكرانيه علي قمه مجموعه الدول الصناعيه السبع في لاهاي   مواضيع قد تهمك نهايه  وأكد الرئيس الأمريكي باراك أوباما خلال لقاء مع نظيره الصيني شي جين بينغ علي أن  واشنطن وبكين يمكنهما  بالعمل سويا  تعزيز القانون الدولي واحترام سياده الدول    وتسيطر قوات روسيه حاليا علي معظم القواعد العسكريه الأوكرانيه في القرم التي أعلنت موسكو ضمها للاتحاد الروسي بعد استفتاء أجرته السلطات المحليه هناك   قلق بالغ  وقال مارك لوين  مراسل بي بي سي في القرم  إن القوات الروسيه تسيطر بشكل كامل علي القاعده  ونقلت الجنود الأوكرانيين بعيدا إلي مكان مجهول   وتعد قاعده فيودوسيا واح

### We notice how the word 'الأوكرانية' was normalized to 'الأوكرانيه'

### Analyze by checking if the text has any diacritics, If so normalize by removing them

In [16]:

# Regular expression to match diacritics in Arabic text
diacritic_pattern = re.compile('[\u064B-\u065F\u0670]')

# Function to count diacritics in a given sentence
def count_diacritics(sentence):
    return len(re.findall(diacritic_pattern, sentence))

# Add new column with diacritic counts for each sentence
train_inputs['Diacritic_Counts'] = train_inputs[0].apply(count_diacritics)
train_labels['Diacritic_Counts'] = train_labels[0].apply(count_diacritics)

# Calculate total diacritic count for all sentences in DataFrame
total_train_diacritic_count = train_inputs['Diacritic_Counts'].sum()
total_label_diacritic_count = train_labels['Diacritic_Counts'].sum()


print(f'Total Training Diacritic Count: {total_train_diacritic_count}')
print(f'Total Lebels Diacritic Count: {total_label_diacritic_count}')



Total Training Diacritic Count: 152867
Total Lebels Diacritic Count: 5966


### After analyzing, we notice how some words in the data have diacritics and some do not. Hence, we need to normalize by removing all diacritics and non-arabic words

In [17]:

# Define a regular expression to match non-Arabic characters
diacritics_and_non_arabic_pattern = re.compile("[^\u0600-\u06FF0-9 ]|[ًٌٍَُِّْ]")

def preprocess_text(text):
    # Remove diacritics and non-Arabic characters from the text
    text = diacritics_and_non_arabic_pattern.sub("", text)

    # Remove Arabic punctuations from the text
    text = ''.join([' ' if ch in punctuations else ch for ch in text])

    return text


# Preprocess the input data
train_inputs['cleaned_text'] = train_inputs[0].apply(preprocess_text)
train_labels['cleaned_text'] = train_labels[0].apply(preprocess_text)


In [18]:
# Calculate total diacritic count for all sentences in DataFrame
total_train_diacritic_count = train_inputs['cleaned_text'].apply(count_diacritics).sum()
total_label_diacritic_count = train_labels['cleaned_text'].apply(count_diacritics).sum()


print(f'Total Training Diacritic Count: {total_train_diacritic_count}')
print(f'Total Labels Diacritic Count: {total_label_diacritic_count}')

Total Training Diacritic Count: 0
Total Labels Diacritic Count: 0


In [19]:
#rename original text column to avoid confusion
train_inputs = train_inputs.rename(columns={0: 'original_Text'})
train_labels = train_labels.rename(columns={0: 'original_Text'})
train_inputs.head()

,original_Text,Punctuation_Counts_before,Diacritic_Counts,cleaned_text
0,وكان الرئيس الأوكراني المؤقت الكسندر تورتشينو...,125,0,وكان الرئيس الأوكراني المؤقت الكسندر تورتشينو...
1,بحلول عام 2050 ستحتاج مصر إلي 21 مليار متر مكع...,70,1,بحلول عام 2050 ستحتاج مصر إلي 21 مليار متر مكع...
2,وذكرت وكاله الأنباء المحليه جي إن إس أن جماع...,35,0,وذكرت وكاله الأنباء المحليه جي إن إس أن جماع...
3,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...,284,15,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...
4,مسلح حوثي في إب وقال المصدر إن المسلحين الحوث...,25,3,مسلح حوثي في إب وقال المصدر إن المسلحين الحوث...


## Removing stop words

### We will remove stop words to ease the training process

In [20]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('arabic'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text
train_inputs['cleaned_text'] = train_inputs['cleaned_text'].apply(remove_stopwords)
# do not remove stop words from labels
# train_labels['cleaned_text'] = train_labels['cleaned_text'].apply(remove_stopwords)


## Lemmatizing tokens

In [21]:
from nltk.stem.isri import ISRIStemmer
def lemmatize_text(text):
    stemmer = ISRIStemmer()
    words = word_tokenize(text)
    lemma_words = [stemmer.suf32(word) for word in words]
    lemma_text = ' '.join(lemma_words)
    return lemma_text


train_inputs['cleaned_text'] = train_inputs['cleaned_text'].apply(lemmatize_text)
train_labels['cleaned_text'] = train_labels['cleaned_text'].apply(lemmatize_text)




In [30]:
train_inputs['cleaned_text'][2]

'وذكرت وكاله الأنباء المحليه جي إس جماعه جيش محمد المتشدده أعلنت مسؤوليت الهجوم منطقه كشمير المتنازع علي الهند وباكس خلال العقود الست الماضيه ظلت منطقه كشمير القريبه جبال الهيمالا محل نزاع الهند وباكس الجنه الملعونه دموع الفقراء كشمير لماذا يلجأ الناس إلي أضرحه الصوفي كشمير الهنديه بالصور الطفوله المسروقه كشمير فمنذ تقسيم الهند وقيام باكس عام وقعت حرب البلد حول منطقه كشمير الأغلبيه المسلمه والتي يطالب البلد بالسياده علي وتعد كشمير اليوم واحده المناطق المدججه بالسلاح العالم تدير الص أجزاء الإقليم تسلسل زمني لأهم الأحداث كشمير'

## Model Architecture
1-An Embedding layer: We need to map the input tokens into word embeddings that the model can understand. <br>
2-An Encoder: this encoder should consist of multiple multi-head attention layers to make use of the attention mechanism and positional encoding to capture the context of the words. <br>
3-Feed-forward layer: This layer should help the model extract features and learn complex non linear relations. <br>
4-Decoder layer: The decoder layer generates the summary text by autoregressively predicting each word in the summary sequence based on the previous words it has generated. This layer typically consists of multiple stacked transformer blocks, each of which includes multi-head self-attention, cross-attention, and feedforward layers. The cross-attention mechanism allows the model to attend to the relevant parts of the input text while generating the summary. <br>
4-Output Layer: This layer produces the final summary text. This layer typically uses a softmax activation function to compute the probabilities of each word in the summary vocabulary, and the word with the highest probability is selected as the next word in the summary sequence. <br>

AraBART is a useful pretrained model that we can use and fine-tune to the summarization task as it already has the above needed layers. Each layer in AraBART is useful for text summarization in different ways:

1-Tokenization layer: AraBART uses a custom Arabic-specific tokenization scheme to tokenize the input text into subwords. This is necessary to handle the rich morphology of Arabic words, which can have multiple affixes and inflections. <br>

2-Embedding layer: The embedding layer maps each subword token in the input sequence to a high-dimensional vector space, where each dimension represents a different feature or aspect of the token. AraBART uses a shared vocabulary for both input and output sequences, which allows the model to share the embeddings between the encoder and decoder. <br>

3-Encoder layer: The encoder layer processes the input sequence and produces a condensed representation of the input text. The encoder in AraBART consists of 12 transformer blocks, each of which includes multi-head self-attention and feedforward layers. The encoder also uses position embeddings to preserve the order of the input sequence. <br>

4-Decoder layer: The decoder layer generates the summary text by autoregressively predicting each subword token in the summary sequence based on the previous subwords it has generated. The decoder in AraBART consists of 12 transformer blocks, each of which includes multi-head self-attention, cross-attention, and feedforward layers. The decoder also uses position embeddings and a mask to ensure that each token is generated based only on the previous tokens. <br>

5-Output layer: The output layer produces the final summary text by selecting the subword token with the highest probability in the summary vocabulary. The output layer in AraBART uses a linear layer followed by a softmax activation function to compute the probabilities of each token in the summary vocabulary. <br>



## Text Tokenization for AraBART

In [31]:
train_data = pd.DataFrame()
train_data['text'] = train_inputs['cleaned_text']
train_data['summary'] = train_labels['cleaned_text']

In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "moussaKam/AraBART"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\_utils.py:146: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  t = torch.tensor([], dtype=storage.dtype, device=storage.untyped().device)


In [56]:
def tokenize_data(df):
    inputs = df['text'].tolist()
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(df['summary'].tolist(), max_length=128, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


In [57]:
tokenized_data = tokenize_data(train_data)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [55]:
print(tokenized_data['input_ids'][0])
print(tokenized_data['labels'][0])


[0, 193, 164, 8694, 14910, 11327, 38570, 8401, 34042, 1473, 560, 16451, 16831, 26255, 9, 18399, 38007, 622, 11345, 9, 178, 13807, 108, 17770, 9, 11511, 9, 28, 32849, 270, 5, 1566, 736, 2722, 9637, 66, 23705, 160, 1051, 1881, 1854, 28, 32849, 18, 44, 1119, 192, 122, 1922, 495, 18848, 9, 561, 26255, 9, 490, 1559, 19284, 192, 22153, 161, 1473, 16831, 1804, 9, 30145, 29632, 9, 37921, 1998, 108, 4807, 26255, 17770, 9, 5, 1566, 736, 10, 20845, 30, 38516, 6103, 2287, 8653, 23597, 7529, 26255, 9, 108, 2000, 9, 14743, 279, 4305, 9, 6170, 34848, 1745, 35701, 25183, 270, 708, 164, 984, 6075, 28, 1680, 66, 960, 4043, 18030, 2008, 8292, 56, 2062, 108, 1372, 308, 50, 7628, 2929, 14324, 1459, 3987, 81, 297, 5979, 192, 9490, 279, 149, 19907, 622, 11345, 9, 446, 108, 946, 5022, 1486, 9, 26255, 9, 18399, 929, 3257, 3722, 17, 3829, 1804, 9236, 25377, 16631, 1484, 9, 5903, 5252, 122, 8324, 311, 29, 4215, 192, 18399, 16831, 1804, 9, 8653, 148, 750, 108, 29632, 9, 4256, 4807, 26255, 1905, 734, 493, 10653, 5

In [1]:
from nltk.stem.isri import ISRIStemmer
def lemmatize_text(text):
    stemmer = ISRIStemmer()
    words = word_tokenize(text)
    lemma_words = [stemmer.suf32(word) for word in words]
    lemma_text = ' '.join(lemma_words)
    return lemma_text

In [4]:
lemmatize_text('وفعلوا')

'وفعل'

In [7]:
lemmatize_text('استسقي')

'استسقي'

In [8]:
def stem_arabic_word(word):
    """
    Stem an Arabic word using the ISRIStemmer from the NLTK library.
    """
    # Initialize the stemmer
    stemmer = ISRIStemmer()

    # Stem the word
    stemmed_word = stemmer.stem(word)

    # Return the stemmed word
    return stemmed_word

In [12]:
stem_arabic_word('استثمر')

'ثمر'